Бэклог:   
- [ ] код из второй статьи запустить
- [ ] как копировать файл с дагом в airflow?

Код из статей:
- [Как запустить Apache AirFlow в Google Colab с ngrok](https://bigdataschool.ru/blog/how-to-launch-airflow-on-colab-with-ngrok.html)
- [Как запустить DAG AirFlow в Google Colab: простой пример](https://bigdataschool.ru/blog/how-to-run-user-dag-airflow-in-colab-with-ngrok.html)

## Как запустить Apache AirFlow в Google Colab с ngrok

In [ ]:
#Установка Apache Airflow и инициализация базы данных
!pip install apache-airflow
!airflow db init
## initdb - было так в оригинале, но пишет об ошибке и замене команды

#Установка инструмента ngrok для создания безопасного туннеля
#для доступа к веб-интерфейсу Airflow из любого места
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!pip install pyngrok

#импорт модулей
from pyngrok import ngrok
import sys
import os
import datetime
from airflow import DAG
from airflow.operators.bash import BashOperator

## подсоединяемся к гугл-диску
from google.colab import drive
drive.mount('/content/drive')
os.makedirs('/content/airflow/dags', exist_ok=True)
dags_folder = os.path.join(os.path.expanduser("~"), "airflow", "dags")
os.makedirs(dags_folder, exist_ok=True)

In [ ]:
#Задание переменной auth_token для аутентификации в сервисе ngrok.
auth_token = "2WhKx82LS1maMJ2dMagXgh9ipsb_3fcJGdvLkWMHihaHRpLju" #@param {type:"string"}
# Since we can't access Colab notebooks IP directly we'll use
# ngrok to create a public URL for the server via a tunnel
# Authenticate ngrok
# https://dashboard.ngrok.com/signup
# Then go to the "Your Authtoken" tab in the sidebar
# and copy the API key
#Аутентификация в сервисе ngrok с помощью auth_token
os.system(f"ngrok authtoken {auth_token}")
#Запуск ngrok, который создаст публичный URL для сервера через туннель
#для доступа к веб-интерфейсу Airflow из любого места.
#addr="8888" указывает на порт, на котором запущен веб-сервер Airflow,
#а proto="http" указывает на использование протокола HTTP
public_url = ngrok.connect(addr="8888", proto="http")
#Вывод публичного URL для доступа к веб-интерфейсу Airflow
print("Адрес Airflow GUI:", public_url)

In [ ]:
## выше адрес вида https://2890-34-69-125-71.ngrok-free.app,
## по которому  зайдем в веб-интерфейс Apache AirFlow

In [ ]:
!airflow db init #Инициализация базы данных Airflow  ## это же было в первой ячейке
!airflow db upgrade #Обновление базы данных Airflow
## !airflow upgradedb - в оригинале было, но ворнинги требуют новый синтаксис
#Создание нового пользователя в Apache Airflow с именем пользователя anna, именем Anna, фамилией Anna, адресом электронной почты anna.doe@example.com и паролем password.
#Этот пользователь будет иметь роль Admin, которая дает полный доступ к интерфейсу Airflow.
!airflow users create --username anna --firstname Anna --lastname Anna --email anna.doe@example.com --role Admin --password password

In [ ]:
#запуск веб-сервера Apache Airflow на порту 8888. Веб-сервер Airflow предоставляет пользовательский интерфейс
#для управления DAG’ами,
#просмотра логов выполнения задач, мониторинга прогресса выполнения
!airflow webserver --port 8888

In [ ]:
## Теперь можно войти в веб-интерфейс Apache AirFlow,
## по ссылке, ранее сгенерированной утилитой ngrok.
## Надо ввести логин и пароль пользователя, созданного ранее: anna, password

In [ ]:
## По завершении работы с веб-интерфейсом AirFlow надо освободить занятый порт
!pkill -f ngrok #поиск и завершение процесса, который запущен с помощью команды ngrok
# отключение публичного URL, созданного с помощью ngrok для веб-сервера Airflow
ngrok.disconnect(public_url=public_url)

-------------

## Как запустить DAG AirFlow в Google Colab: простой пример

In [ ]:
#Установка Apache Airflow и инициализация базы данных.
!pip install apache-airflow
## !airflow init  ## еще будет в следующей ячейке
## initdb - было так в оригинале, но пишет об ошибке и замене команды

#Установка инструмента ngrok для создания безопасного туннеля для доступа к веб-интерфейсу Airflow из любого места
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!pip install pyngrok

## !pip install sqlalchemy
## установка ругается на конфликт со старой версией алхимии
## но отдельная установка в лоб не проходит: уже установлено

#импорт модулей
from pyngrok import ngrok
import sys
import os
import datetime
from airflow import DAG
from airflow.operators.bash import BashOperator

## свой dag будем хранить на гугл-драйв
from google.colab import drive
drive.mount('/content/drive')
os.makedirs('/content/airflow/dags', exist_ok=True)
dags_folder = os.path.join(os.path.expanduser("~"), "airflow", "dags")
os.makedirs(dags_folder, exist_ok=True)

In [ ]:
#Задание переменной auth_token для аутентификации в сервисе ngrok.
auth_token = "2WhKx82LS1maMJ2dMagXgh9ipsb_3fcJGdvLkWMHihaHRpLju"
# Since we can't access Colab notebooks IP directly we'll use
# ngrok to create a public URL for the server via a tunnel
# Authenticate ngrok
# https://dashboard.ngrok.com/signup
# Then go to the "Your Authtoken" tab in the sidebar and copy the API key
#Аутентификация в сервисе ngrok с помощью auth_token
os.system(f"ngrok authtoken {auth_token}")
#Запуск ngrok, который создаст публичный URL для сервера через туннель
#для доступа к веб-интерфейсу Airflow из любого места.
#addr="8888" указывает на порт, на котором запущен веб-сервер Airflow, 
#а proto="http" указывает на использование протокола HTTP
public_url = ngrok.connect(addr="8888", proto="http")
#Вывод публичного URL для доступа к веб-интерфейсу Airflow

In [ ]:
## ip для входа чуть выше в формате типа https://ec27-34-86-112-48.ngrok-free.app

In [ ]:
!airflow db init #Инициализация базы данных Airflow
## это же было в первой ячейке
## но без этого не подгружаются примеры?

!airflow db upgrade #Обновление базы данных Airflow
## !airflow upgradedb - в оригинале было, но ворнинги требуют новый синтаксис

#Создание нового пользователя в Apache Airflow
#Этот пользователь будет иметь роль Admin, которая дает полный доступ к интерфейсу Airflow.
!airflow users create --username boris --firstname Boris --lastname Yu --email gbrs@example.com --role Admin --password password

In [ ]:
## копируем файл с дагом в airflow
!cp /content/airflow/dags/BORIS_DAG_SaveFile.py ~/airflow/dags/BORIS_DAG_SaveFile.py
## ?
!airflow dags unpause BORIS_DAG_SaveFile

In [ ]:
## запуск веб-сервера Airflow на порту 8888 в фоновом режиме (daemon mode)
!airflow webserver --port 8888 --daemon

In [ ]:
## запустить планировщик AirFlow
!airflow scheduler --daemon

In [ ]:
print("Адрес Airflow GUI:", public_url)

можно открыть веб-браузер и перейти на страницу URL-адреса выше (полученного ранее с помощью утилиты ngrok, которая перебрасывает локальный хост http://localhost:8888 на внешний URL-адрес).  
Теперь можно войти в веб-интерфейс Apache AirFlow, используя логин и пароль ранее созданного пользователя.

In [ ]:
## По завершении работы с веб-интерфейсом AirFlow надо освободить занятый порт
!pkill -f ngrok #поиск и завершение процесса, который запущен с помощью команды ngrok
# отключение публичного URL, созданного с помощью ngrok для веб-сервера Airflow
ngrok.disconnect(public_url=public_url)